# Spectrum

In [1]:
import pandas as pd
import astropy.units as u
import numpy as np
import matplotlib.pyplot as plt
from astroquery.simbad import Simbad
from astropy.coordinates import SkyCoord
from pandas.plotting import scatter_matrix

In [2]:
catalog = pd.read_csv("markeddata/gevtevsimbad.txt", sep='\t', index_col=0)

In [3]:
catalog.head()

,level_0,gev_0FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,...,tev_spec_pl_norm_err_sys,tev_spec_theta,tev_spec_type,tev_tevcat2_id,tev_tevcat_id,tev_tevcat_name,tev_tgevcat_id,tev_tgevcat_name,tev_where,marked
0,79,b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',b' ',...,NaN,NaN,b'none',b'PehQOM',121,b'TeV J1745-290d',107,b'TeV J1745-2900',b'gal',0
1,81,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,NaN,0.09,b'pl',b'LsZtAg',277,b'TeV J1746-289',-9223372036854775808,b'',b'gal',0
2,108,b' ',b'1FGL J1746.4-2849c',b'1FHL J1746.3-2851 ',b'2FGL J1746.6-2851c',b'PWN G0.13-0.11 ',b' ',b' ',b'3EG J1746-2851',b' ',...,NaN,NaN,b'none',b'LsZtAg',269,b'TeV J1746-289',-9223372036854775808,b'',b'gal',0
3,97,b'0FGL J2032.2+4122',b'1FGL J2032.2+4127 ',b'1FHL J2032.1+4125 ',b'2FGL J2032.2+4126 ',b'LAT PSR J2032+4127 ',b' ',b'1AGL J2032+4102',b' ',b' ',...,2.200000e-13,NaN,b'pl',b'3IY6Gd',87,b'TeV J2032+415',146,b'TeV J2031+4133',b'gal',1
4,37,b'0FGL J1018.2-5858',b'1FGL J1018.6-5856 ',b'1FHL J1018.9-5855 ',b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b' ',b' ',b' ',b' ',...,5.800000e-14,NaN,b'pl',b'5FlzLA',237,b'TeV J1018-589',45,b'TeV J1018-5856',b'gal',1


Columns contained spectrum in gev:

In [4]:
gev_spectrum_columns = [
    'gev_Flux1000',
    'gev_Flux10000_100000',
    'gev_Flux1000_3000',
    'gev_Flux100_300',
    'gev_Flux3000_10000',
    'gev_Flux300_1000',
    'gev_Flux30_100',
]

Columns contained spectrum in simbad:

In [5]:
s_spectrum_columns = [
    's_FLUX_B', 
    's_FLUX_G', 
    's_FLUX_H', 
    's_FLUX_J', 
    's_FLUX_K', 
    's_FLUX_U', 
    's_FLUX_U_1', 
    's_FLUX_V',
    ]

In [6]:
gevoptics_spectrum = catalog[gev_spectrum_columns + s_spectrum_columns].copy()

In [7]:
display(catalog[gev_spectrum_columns + s_spectrum_columns + ['s_MAIN_ID']])

,gev_Flux1000,gev_Flux10000_100000,gev_Flux1000_3000,gev_Flux100_300,gev_Flux3000_10000,gev_Flux300_1000,gev_Flux30_100,s_FLUX_B,s_FLUX_G,s_FLUX_H,s_FLUX_J,s_FLUX_K,s_FLUX_U,s_FLUX_U_1,s_FLUX_V,s_MAIN_ID
0,3.874533e-08,3.055154e-10,3.184363e-08,1.613918e-08,5.155299e-09,9.678488e-08,NaN,NaN,NaN,14.510,NaN,10.490,NaN,NaN,NaN,b'GCIRS 8'
1,2.752177e-08,4.572024e-10,2.235459e-08,1.851010e-09,4.686388e-09,4.536945e-08,NaN,NaN,NaN,10.687,13.337,9.122,NaN,NaN,NaN,b'[MCD2010] 13'
2,2.752177e-08,4.572024e-10,2.235459e-08,1.851010e-09,4.686388e-09,4.536945e-08,NaN,NaN,NaN,10.687,13.337,9.122,NaN,NaN,NaN,b'[MCD2010] 13'
3,2.458654e-08,3.071351e-10,1.885649e-08,5.618088e-08,5.075542e-09,4.394119e-08,NaN,12.010,10.9776,8.895,9.191,8.766,12.390000,12.390000,11.300000,b'Schulte 21'
4,2.713704e-08,1.420659e-10,2.192043e-08,2.666705e-07,3.261212e-09,1.141154e-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b'2FGL J1019.0-5856'
5,1.184660e-08,1.087441e-09,8.559550e-09,5.170882e-09,3.236763e-09,1.076956e-08,NaN,NaN,NaN,NaN,NaN,8.900,NaN,NaN,NaN,b'[KRL2007b] 332'
6,1.909024e-08,2.244687e-10,1.828320e-08,4.639998e-07,1.954995e-09,1.100878e-07,NaN,12.230,10.7988,8.751,9.024,8.530,12.020000,12.020000,11.270000,b'V* V479 Sct'
7,4.606472e-08,5.130483e-10,3.871007e-08,5.552702e-07,6.510779e-09,1.962727e-07,NaN,11.610,10.3933,8.228,8.621,7.917,11.270000,11.270000,10.750000,b'LS I +61 303'
8,9.343741e-10,5.395927e-11,9.321550e-10,7.792275e-09,1.366562e-10,2.611374e-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,b'CXOU J095550.1+694034'
9,2.178238e-08,9.998126e-10,1.540335e-08,5.515597e-07,5.319086e-09,8.277984e-08,NaN,NaN,NaN,13.100,15.555,10.700,NaN,NaN,NaN,b'SSTGC 519103'


In [8]:
gevoptics_spectrum = gevoptics_spectrum.fillna(0)

Normilization:

In [9]:
gevoptics_spectrum = gevoptics_spectrum.div(gevoptics_spectrum.sum(axis=1), axis=0)

In [10]:
gevoptics_spectrum['marked'] = catalog['marked']

In [11]:
colors=['red','black']
c=gevoptics_spectrum['marked'].apply(lambda x:colors[x])

In [12]:
scatter_matrix(gevoptics_spectrum,figsize=[40,40],marker='o',c=c)
plt.savefig('plot.png')

C:\Users\Admin\Anaconda3\lib\site-packages\matplotlib\axes\_base.py:2918: UserWarning: Attempting to set identical left==right results
in singular transformations; automatically expanding.
left=0.0, right=0.0
  'left=%s, right=%s') % (left, right))
C:\Users\Admin\Anaconda3\lib\site-packages\matplotlib\axes\_base.py:3239: UserWarning: Attempting to set identical bottom==top results
in singular transformations; automatically expanding.
bottom=0.0, top=0.0
  'bottom=%s, top=%s') % (bottom, top))
